In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

### Sequential chain -- LLMChain

In [5]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
marketing_text = PromptTemplate(
    input_variables = ['name','description'],
    template="Generate a one line facebook ad copy for a product called {name}. Below is a description of this product: {description}"
)

In [8]:
text_llm = GoogleGenerativeAI(model='gemini-1.5-pro-001')

In [9]:
text_chain = LLMChain(llm=text_llm,prompt=marketing_text,output_key="copy_text")

C:\Users\Dell\AppData\Local\Temp\ipykernel_892\870116488.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  text_chain = LLMChain(llm=text_llm,prompt=marketing_text,output_key="copy_text")


In [10]:
translate_text = PromptTemplate(input_variables=["copy_text"],template="""Translate this text to Hindi:{copy_text}""")

In [11]:
translate_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [12]:
translate_chain = LLMChain(llm=translate_llm,prompt=translate_text,output_key="translated_copy")

In [13]:
seq_chain = SequentialChain(
    chains = [text_chain,translate_chain],
    input_variables=["name","description"],
    output_variables=["copy_text","translated_copy"]
)

In [15]:
message = seq_chain({"name":"AeroGlow Nightlight","description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling."})

In [16]:
print(message)

{'name': 'AeroGlow Nightlight', 'description': 'AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling.', 'copy_text': '✨ Say "Goodnight" to darkness! ✨ Introducing AeroGlow Nightlight, the voice-activated nightlight that paints your ceiling with calming constellations. ', 'translated_copy': 'Here\'s the translation of the text into Hindi:\n\n✨ अंधेरे को "शुभ रात्रि" कहें! ✨ पेश करते हैं AeroGlow नाइटलाइट, एक आवाज से चलने वाली नाइटलाइट जो आपकी छत को शांत नक्षत्रों से रंग देती है।\n\nHere\'s a slightly different, more evocative translation:\n\n✨ अब अंधेरे को कहिए "अलविदा"! ✨ पेश है AeroGlow नाइटलाइट, जो आपकी आवाज़ से रोशन होती है और आपकी छत पर शांत नक्षत्रों का जादू बिखेरती है।'}
